<a href="https://colab.research.google.com/github/veshitala/Deep_learning_experiments/blob/master/Importnat_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HOW **add Google drive into colab**

In [0]:
#Start by connecting gdrive into the google colab
from google.colab import drive
drive.mount('/content/gdrive')

**How to download the required packages to save the model**

In [0]:
!pip install h5py pyyaml
!pip install tf_nightly

# How to save your model in h5 format

In [0]:
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

**How to load a saved model**

In [0]:
import tensorflow as tf 
from keras.models import load_model
classifierLoad = tf.keras.models.load_model('my_model.h5')

# How to check and upload images to test your model

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes)
  if classes[0]>0.5:
    print(fn + " is a notmonkey")
  else:
    print(fn + " is a monkey")

# How to give path during saving the model

In [0]:
model_save_name = 'transfer.h5'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.save(path)

# Wtaching how your images are being transformed through each layer

In [0]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# Let's prepare a random input image from the training set.
"""monkey_img_files = [os.path.join(train_monkey_dir, f) for f in train_monkey_names]
notmonkey_img_files = [os.path.join(train_notmonkey_dir, f) for f in train_notmonkey_names]
img_path = random.choice(monkey_img_files + notmonkey_img_files)"""
img_path = F"/content/monkey.25.jpg"
img = load_img(img_path, target_size=(300, 300))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')